In [1]:
import json
from weaviate.classes.query import Filter
import weaviate
import joblib
import pandas as pd
import flask_app
import weaviate_server
import unittests
import json
from utils import (
    ChatWidget, 
    generate_with_single_input,
    parse_json_output,
    get_filter_by_metadata,
    generate_filters_from_query,
    process_and_print_query,
    print_properties,
    make_url
)

 * Serving Flask app 'flask_app'
 * Debug mode: off


In [2]:
client = weaviate.connect_to_local(port=8079, grpc_port=50050)

In [3]:
import phoenix as px
from phoenix.otel import register
from opentelemetry.trace import Status, StatusCode
# Launch the lab and the URL
make_url()
session = px.launch_app()

/opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:319: PydanticDeprecatedSince20: `json_encoders` is deprecated. See https://docs.pydantic.dev/2.12/concepts/serialization/#custom-serializers for alternatives. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.11/site-packages/alembic/config.py:598: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(
/opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.11/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_cumulative_llm_token_count_total
  next(self.gen)
/opt/homebrew/Caskroom/miniforge/base/envs/myen

FOLLOW THIS URL TO OPEN THE UI: http://127.0.0.1:8888
🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix


In [4]:
# Setting up the telemetry

phoenix_project_name = "chatbot"

# With phoenix, we just need to register to get the tracer provider with the appropriate endpoint. 
# Different from the ungraded lab, you will NOT use auto_instrument = True, as there are LLM calls not needed to be traced (examples, calls within unittests etc.)

tracer_provider_phoenix = register(project_name=phoenix_project_name, endpoint="http://127.0.0.1:6006/v1/traces")

# Retrieve a tracer for manual instrumentation
tracer = tracer_provider_phoenix.get_tracer(__name__)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: chatbot
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://127.0.0.1:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [5]:
# Loading products data
products_data = joblib.load('dataset/clothes_json.joblib')
# Let's get one example
products_data[0]
faq = joblib.load("dataset/faq.joblib")
# Get an example
faq[:2]

[{'question': 'What are your store hours?',
  'answer': 'Our online store is open 24/7. Customer service is available from 9:00 AM to 6:00 PM, Monday through Friday.',
  'type': 'general information'},
 {'question': 'Where is Fashion Forward Hub located?',
  'answer': 'Fashion Forward Hub is primarily an online store. Our corporate office is located at 123 Fashion Lane, Trend City, Style State.',
  'type': 'general information'}]

In [6]:
products_data[:2]

[{'gender': 'Men',
  'masterCategory': 'Apparel',
  'subCategory': 'Topwear',
  'articleType': 'Shirts',
  'baseColour': 'Navy Blue',
  'season': 'Fall',
  'year': 2011,
  'usage': 'Casual',
  'productDisplayName': 'Turtle Check Men Navy Blue Shirt',
  'price': 67.0,
  'product_id': 15970},
 {'gender': 'Men',
  'masterCategory': 'Apparel',
  'subCategory': 'Bottomwear',
  'articleType': 'Jeans',
  'baseColour': 'Blue',
  'season': 'Summer',
  'year': 2012,
  'usage': 'Casual',
  'productDisplayName': 'Peter England Men Party Blue Jeans',
  'price': 22.0,
  'product_id': 39386}]

In [7]:
# The output is a dictionary containing the role and content from the LLM call, as well as the token usage.:
result = generate_with_single_input("What are the primary colors?")
print(json.dumps(result, indent = 2))

{
  "id": "chatcmpl-CYrs4SBn8HBLk7GNXhwT2wM5Kb8CK",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The primary colors can refer to different sets depending on the color model being discussed:\n\n1. **Additive Color Model (RGB)**: In this model, used for light and digital screens, the primary colors are:\n   - Red\n   - Green\n   - Blue\n\nWhen combined in different ways, these colors can create a wide range of other colors, including white when all are combined at full intensity.\n\n2. **Subtractive Color Model (CMY/CMYK)**: In this model, used in color printing, the primary colors are:\n   - Cyan\n   - Magenta\n   - Yellow\n   - (Black, represented as \"K\" in CMYK, is often added for depth and detail in printing.)\n\n3. **Traditional Artist's Colors (RYB)**: In the context of art and traditional color mixing, the primary colors are often considered to be:\n   - Red\n   - Yellow\n   - Blue\n\nThese a

In [8]:
# To retreive the content, then you should do as follows:
print(result['choices'][0]['message']['content'])

The primary colors can refer to different sets depending on the color model being discussed:

1. **Additive Color Model (RGB)**: In this model, used for light and digital screens, the primary colors are:
   - Red
   - Green
   - Blue

When combined in different ways, these colors can create a wide range of other colors, including white when all are combined at full intensity.

2. **Subtractive Color Model (CMY/CMYK)**: In this model, used in color printing, the primary colors are:
   - Cyan
   - Magenta
   - Yellow
   - (Black, represented as "K" in CMYK, is often added for depth and detail in printing.)

3. **Traditional Artist's Colors (RYB)**: In the context of art and traditional color mixing, the primary colors are often considered to be:
   - Red
   - Yellow
   - Blue

These are used in painting and can be mixed to create a vast array of other colors.

Each of these models serves different purposes and contexts for color mixing and representation.


In [9]:
# The total tokens count (input + output) for this is:
print(result['usage']['total_tokens'])

237


In [10]:
def generate_params_dict(
    prompt: str,
    temperature: float = 1.0,
    role: str = 'user',
    top_p: float = 1.0,
    max_tokens: int = 500,
    model: str = "gpt-4o-mini"
) -> dict:
    """
    Generates a dictionary of parameters for calling a Language Learning Model (LLM),
    allowing for the customization of several key options that can affect the output from the model. 

    Args:
        prompt (str): The input text that will be provided to the model to guide text generation.
        temperature (float): A value between 0 and 1 that controls the randomness of the model's output; 
            lower values result in more repetitive and deterministic results, while higher values enhance randomness.
        role (str): The role designation to be used in context, typically identifying the initiator of the interaction.
        top_p (float): A value between 0 and 1 that manages diversity through the technique of nucleus sampling; 
            this parameter limits the set of considered words to the smallest possible while maintaining 'top_p' cumulative probability.
        max_tokens (int): The maximum number of tokens that the model is allowed to generate in response, where a token can 
            be as short as one character or as long as one word.
        model (str): The specific model identifier to be utilized for processing the request. This typically specifies both 
            the version and configuration of the LLM to be employed.

    Returns:
        dict: A dictionary containing all specified parameters which can then be used to configure and execute a call to the LLM.
    """
    # Create the dictionary with the necessary parameters
    kwargs = {
        "prompt": prompt,
        "role": role,
        "temperature": temperature,
        "top_p": top_p,
        "max_tokens": max_tokens,
        "model": model
    }
    return kwargs

In [11]:
def generate_params_dict(
    prompt: str,
    temperature: float = 1.0,
    role: str = 'user',
    top_p: float = 1.0,
    max_tokens: int = 500,
    model: str = "gpt-4o-mini"
) -> dict:
    """
    Generates a dictionary of parameters for calling a Language Learning Model (LLM),
    allowing for the customization of several key options that can affect the output from the model. 

    Args:
        prompt (str): The input text that will be provided to the model to guide text generation.
        temperature (float): A value between 0 and 1 that controls the randomness of the model's output; 
            lower values result in more repetitive and deterministic results, while higher values enhance randomness.
        role (str): The role designation to be used in context, typically identifying the initiator of the interaction.
        top_p (float): A value between 0 and 1 that manages diversity through the technique of nucleus sampling; 
            this parameter limits the set of considered words to the smallest possible while maintaining 'top_p' cumulative probability.
        max_tokens (int): The maximum number of tokens that the model is allowed to generate in response, where a token can 
            be as short as one character or as long as one word.
        model (str): The specific model identifier to be utilized for processing the request. This typically specifies both 
            the version and configuration of the LLM to be employed.

    Returns:
        dict: A dictionary containing all specified parameters which can then be used to configure and execute a call to the LLM.
    """
    # Create the dictionary with the necessary parameters
    kwargs = {
        "prompt": prompt,
        "role": role,
        "temperature": temperature,
        "top_p": top_p,
        "max_tokens": max_tokens,
        "model": model
    }
    return kwargs

In [12]:
kwargs = generate_params_dict("Solve 3x^2 + 5 = 0")
print(kwargs)
# Now you can call the LLM 
result = generate_with_single_input(**kwargs)
content = result['choices'][0]['message']['content']
total_tokens = result['usage']['total_tokens']
print(f"Content: {content}\n\nTotal Tokens: {total_tokens}")

{'prompt': 'Solve 3x^2 + 5 = 0', 'role': 'user', 'temperature': 1.0, 'top_p': 1.0, 'max_tokens': 500, 'model': 'gpt-4o-mini'}
Content: To solve the equation \( 3x^2 + 5 = 0 \), we first isolate the \( x^2 \) term:

1. Subtract 5 from both sides:
   \[
   3x^2 = -5
   \]

2. Divide both sides by 3:
   \[
   x^2 = -\frac{5}{3}
   \]

Since the right side is negative, \( x^2 \) is equal to a negative number. This means there are no real solutions to this equation. However, we can find complex solutions by taking the square root:

3. Taking the square root of both sides:
   \[
   x = \pm \sqrt{-\frac{5}{3}} = \pm i \sqrt{\frac{5}{3}}
   \]

4. We can express the final answer as:
   \[
   x = \pm \frac{i\sqrt{15}}{3}
   \]

Thus, the solutions to the equation \( 3x^2 + 5 = 0 \) are:
\[
x = \frac{i\sqrt{15}}{3} \quad \text{and} \quad x = -\frac{i\sqrt{15}}{3}
\]

Total Tokens: 286


In [13]:
# GRADED CELL 
def check_if_faq_or_product(query, simplified = False):
    """
    Determines whether a given instruction prompt is related to a frequently asked question (FAQ) or a product inquiry.

    Parameters:
    - query (str): The instruction or query that needs to be labeled as either FAQ or Product related.
    - simplified (bool): If True, uses a simplified prompt.

    Returns:
    - str: The label 'FAQ' if the prompt is deemed a frequently asked question, 'Product' if it is related to product information, or
      None if the label is inconclusive.
    """
 
    # If not simplified, uses a more complex prompt
    if not simplified:
        PROMPT = f"""Label the following instruction as an FAQ related answer or a product related answer for a clothing store.
        Product related answers are answers specific about product information or that needs to use the products to give an answer.
        Examples:
                Is there a refund for incorrectly bought clothes? Label: FAQ
                Where are your stores located?: Label: FAQ
                Tell me about the cheapest T-shirts that you have. Label: Product
                Do you have blue T-shirts under 100 dollars? Label: Product
                What are the available sizes for the t-shirts? Label: FAQ
                How can I contact you via phone? Label: FAQ
                How can I find the promotions? Label: FAQ
                Give me ideas for a sunny look. Label: Product
        Return only one of the two labels: FAQ or Product, nothing more.
        Query to classify: {query}
                 """

    ##############################################
    ######### GRADED PART STARTS HERE ############
    ##############################################
    
    ### START CODE HERE ###

    # If simlpified, uses a simplified prompt.
    else:
        PROMPT = f"""
                    Clothing store router: label the user message as **FAQ** or **Product**. Output exactly one word.

FAQ = store policies/info and general size/availability questions.

Product = requests to find/compare specific items in the catalog or styling advice using our items.

Examples:
Refund for incorrectly bought clothes → FAQ
Give me ideas for a sunny look → Product

User message: {query}

                 """

        
    ### END CODE HERE ###

    ##############################################
    ######### GRADED PART ENDS HERE ############
    ##############################################
        
    with tracer.start_as_current_span("routing_faq_or_product", openinference_span_kind = 'tool') as span:
        span.set_input(str({"query":query, "simplified": simplified}))
        
        # Get the kwargs dictinary to call the llm, with PROMPT as prompt, low temperature (0 or near 0) and max_tokens = 10
        kwargs = generate_params_dict(PROMPT, temperature = 0, max_tokens = 10)

        # Call generate_with_single_input with **kwargs
        with tracer.start_as_current_span("router_call", openinference_span_kind = 'llm') as router_span:
            router_span.set_input(kwargs)
            try:
                response = generate_with_single_input(**kwargs) 
            except Exception as error:
                router_span.record_exception(error)
                router_span.set_status(Status(StatusCode.ERROR))
            else:
                # OpenInference Semantic Conventions for computing Costs
                router_span.set_attribute("llm.token_count.prompt", response['usage']['prompt_tokens'])
                router_span.set_attribute("llm.token_count.completion", response['usage']['completion_tokens'])
                router_span.set_attribute("llm.token_count.total", response['usage']['total_tokens'])
                router_span.set_attribute("llm.model_name", response['model'])
                router_span.set_attribute("llm.provider", 'together.ai')
                router_span.set_output(response)
                router_span.set_status(Status(StatusCode.OK))
        
    
        # Get the Label by accessing the content key of the response dictionary
        label = response['choices'][0]['message']['content']
        total_tokens = response['usage']['total_tokens']
        span.set_output(str({"label": label, 'total_tokens':total_tokens}))
        span.set_status(Status(StatusCode.OK))

        # Improvement to prevent cases where LLM outputs more than one word
        if 'faq' in label.lower():
            label = 'FAQ'
        elif 'product' in label.lower():
            label = 'Product'
        else:
            label = 'undefined'
    
        return label, total_tokens

In [14]:
unittests.test_check_if_faq_or_product(check_if_faq_or_product)

 All tests passed!


In [15]:
queries = [
    'What is your return policy?', 
    'Give me three examples of blue T-shirts you have available.', 
    'How can I contact the user support?', 
    'Do you have blue Dresses?',
    'Create a look suitable for a wedding party happening during dawn.'
]

labels = ['FAQ', 'Product', 'FAQ', 'Product', 'Product']

for query, correct_label in zip(queries, labels):
    # Call check_if_faq_or_product and store the results
    response_std, tokens_std = check_if_faq_or_product(query, simplified=False)
    response_simp, tokens_simp = check_if_faq_or_product(query, simplified=True)
    
    # Print results
    process_and_print_query(query, correct_label, response_std, tokens_std, response_simp, tokens_simp)

Query: What is your return policy?
  Standard    → Label: FAQ | Tokens: 190
  Simplified  → Label: FAQ | Tokens: 93

Query: Give me three examples of blue T-shirts you have available.
  Standard    → Label: Product | Tokens: 196
  Simplified  → Label: Product | Tokens: 99

Query: How can I contact the user support?
  Standard    → Label: FAQ | Tokens: 192
  Simplified  → Label: FAQ | Tokens: 95

Query: Do you have blue Dresses?
  Standard    → Label: Product | Tokens: 190
  Simplified  → Label: Product | Tokens: 93

Query: Create a look suitable for a wedding party happening during dawn.
  Standard    → Label: Product | Tokens: 196
  Simplified  → Label: Product | Tokens: 99



In [16]:
@tracer.tool
def generate_faq_layout(faq_dict):
    """
    Generates a formatted string layout for a list of FAQs.

    This function iterates through a dictionary of frequently asked questions (FAQs) and constructs
    a string where each question is followed by its corresponding answer and type.

    Parameters:
    - faq_dict (list): A list of dictionaries, each containing keys 'question', 'answer', and 'type' 
      representing an FAQ entry.

    Returns:
    - str: A string representing the formatted layout of FAQs, with each entry on a separate line.
    """
    # Initialize an empty string
    t = ""
    
    # Iterate over every FAQ question in the FAQ list
    for f in faq_dict:
        # Append the question with formatted string (remember to use f-string and access the values as f['question'], f['answer'] and so on)
        # Also, do not forget to add a new line character (\n) at the end of each line.
        t += f"Question: {f['question']} Answer: {f['answer']} Type: {f['type']}\n" 

    return t

In [17]:
# You can generate a full faq_layout with the entire FAQ questions
faq_layout = generate_faq_layout(faq)
print(faq_layout[:1000])

Question: What are your store hours? Answer: Our online store is open 24/7. Customer service is available from 9:00 AM to 6:00 PM, Monday through Friday. Type: general information
Question: Where is Fashion Forward Hub located? Answer: Fashion Forward Hub is primarily an online store. Our corporate office is located at 123 Fashion Lane, Trend City, Style State. Type: general information
Question: Do you have a physical store location? Answer: At this time, we operate exclusively online. This allows us to offer a broader selection and lower prices directly to you. Type: general information
Question: How can I create an account with Fashion Forward Hub? Answer: Click on 'Sign Up' in the top right corner of our website and follow the instructions to set up your account. Type: general information
Question: How do I subscribe to your newsletter? Answer: To receive the latest updates and promotions, sign up for our newsletter at the bottom of our homepage. Type: general information
Question:

In [18]:
# You can choose some faq questions and generate a layout from them. 
# They just need to be in a list with dictionaries with the necessary keys: 'question', 'answer' and 'type'
print(generate_faq_layout(faq[1:2]))

Question: Where is Fashion Forward Hub located? Answer: Fashion Forward Hub is primarily an online store. Our corporate office is located at 123 Fashion Lane, Trend City, Style State. Type: general information



In [19]:
faq_collection = client.collections.get("Faq")

In [20]:
from tqdm import tqdm
from weaviate.util import generate_uuid5
# Set up a batch process with specified fixed size and concurrency
with faq_collection.batch.fixed_size(batch_size=20, concurrent_requests=5) as batch:
    # Iterate over a subset of the dataset
    for document in tqdm(faq):
        # Generate a UUID based on the chunk text for unique identification
        uuid = generate_uuid5(document['question'])

        # Add the chunk object to the batch with properties and UUID
        batch.add_object(
            properties=document,
            uuid=uuid,
        )

100%|████████████| 25/25 [00:00<00:00, 17243.48it/s]


In [21]:
res = faq_collection.query.near_text("What is the return policy?", limit = 5)

In [22]:
for obj in res.objects:
    print_properties(obj)

{
  "answer": "Material information is detailed on each product page under 'Product Details.'",
  "question": "What materials are used in your products?",
  "type": "product information"
}
{
  "answer": "Currently, we only accept orders placed through our website for security reasons.",
  "question": "Can I place an order over the phone?",
  "type": "general information"
}
{
  "answer": "Our inventory is updated regularly with new arrivals each season. Check back frequently for the latest additions.",
  "question": "How often do you update your inventory?",
  "type": "product information"
}
{
  "answer": "Yes, sign up for restock notifications on the product page by providing your email.",
  "question": "Can I get notified when an item is back in stock?",
  "type": "product information"
}
{
  "answer": "We accept returns within 30 days of delivery. Conditions apply for specific categories like accessories.",
  "question": "What is your return policy timeframe?",
  "type": "returns and 

In [23]:
# GRADED CELL 

def query_on_faq(query, simplified = False, **kwargs):
    """
    Constructs a prompt to query an FAQ system and generates a response.

    This function integrates an FAQ layout into the prompt to help generate a suitable answer to the given query
    using a language model. It supports additional keyword arguments to customize the prompt generation process.

    Parameters:
    - query (str): The query about which the function seeks to provide an answer from the FAQ.
    - simplified (bool): If True, uses semantic search to extract a relevant subset of FAQ questions
    - **kwargs: Optional keyword arguments for extra configuration of prompt parameters.

    Returns:
    - str: The response generated from the language model based on the input query and FAQ layout.

    """

    
    # If not simplified, generate the faq layout with the entire FAQ questions
    if not simplified:
        # Set the tracer as a chain type, since in non-simplified version, the full FAQ is used
        with tracer.start_as_current_span("query_on_faq", openinference_span_kind="tool") as span:
            
            span.set_input({"query": query, "simplified": simplified})
            faq_layout = generate_faq_layout(faq)
            
            # Generate the prompt
            PROMPT = f"""You will be provided with an FAQ for a clothing store. 
        Answer the instruction based on it. You might use more than one question and answer to make your answer. Only answer the question and do not mention that you have access to a FAQ. 
        <FAQ_ITEMS>
        PROVIDED FAQ: {faq_layout}
        </FAQ_ITEMS>
        Question: {query}
            """ 
            span.set_attribute("prompt", PROMPT)

            # Generate the parameters dict with PROMPT and **kwargs 
            kwargs = generate_params_dict(PROMPT, **kwargs) 

            span.set_attribute("output", str(kwargs))
            span.set_status(Status(StatusCode.OK))
    
            return kwargs
        
   
    
    else:
        with tracer.start_as_current_span("query_on_faq", openinference_span_kind="tool") as span:
            span.set_input({"query": query, "simplified": simplified})
            with tracer.start_as_current_span("retrieve_faq_questions", openinference_span_kind="retriever") as retrieve_span:
                
                ##############################################
                ######### GRADED PART STARTS HERE ############
                ##############################################
                
                ### START CODE HERE ###
                
                # Get the 5 most relevant FAQ objects, in this case limit = 5
                results = faq_collection.query.near_text(query, limit=5)

                ### END CODE HERE ###

                ##############################################
                ######### GRADED PART ENDS HERE ##############
                ##############################################
                
                # Set the retrieved documents as attributes on the span
                for i, document in enumerate(results.objects): 
                    retrieve_span.set_attribute(f"retrieval.documents.{i}.document.id", str(document.uuid)) 
                    retrieve_span.set_attribute(f"retrieval.documents.{i}.document.metadata", str(document.metadata)) 
                    retrieve_span.set_attribute( 
                        f"retrieval.documents.{i}.document.content", str(document.properties) 
                    )  
            # Transform the results in a list of dictionary
                results = [x.properties for x in results.objects] 
                # Reverse the order to add the most relevant objects in the bottom, so it gets closer to the end of the input
                results.reverse() 
                # Generate the faq layout with the new list of FAQ questions `results`
                faq_layout = generate_faq_layout(results) 

            # Different prompt to deal with this new scenario. 
            PROMPT = (f"You will be provided with a query for a clothing store regarding FAQ. It will be provided relevant FAQ from the clothing store." 
        f"Answer the query based on the relevant FAQ provided. They are ordered in decreasing relevance, so the first is the most relevant FAQ and the last is the least relevant."  
        f"Answer the instruction based on them. You might use more than one question and answer to make your answer. Only answer the question and do not mention that you have access to a FAQ.\n"  
        f"<FAQ>\n"  
        f"RELEVANT FAQ ITEMS:\n{faq_layout}\n"  
        f"</FAQ>\n" 
        f"Query: {query}")

    
        
            span.set_attribute("prompt", PROMPT)
        
            # Generate the parameters dict with PROMPT and **kwargs 
            kwargs = generate_params_dict(PROMPT, **kwargs) 
        
            span.set_attribute("output", str(kwargs))
            span.set_status(Status(StatusCode.OK))
    
            return kwargs

In [24]:
unittests.test_query_on_faq(query_on_faq)

 All tests passed!


In [25]:
# Get the dictionary of arguments
kwargs = query_on_faq("I received the dress I ordered but I don't like it. How can I return it?")

In [26]:
# The number of split tokens in this prompt is:
print(len(kwargs['prompt'].split()))

788


In [27]:
# Run the inference
result = generate_with_single_input(**kwargs)

In [28]:
print(result['choices'][0]['message']['content'])

You can return the dress within 30 days of delivery. To initiate the return, use our Returns Center to select the item you wish to return. We provide a prepaid return label for domestic returns.


In [29]:
# Get the total tokens
print(result['usage']['total_tokens'])

1064


In [30]:
# Get the dictionary of arguments
kwargs = query_on_faq("I received the dress I ordered but I don't like it. How can I return it?", simplified = True)
# The number of split tokens in this prompt is:
print(len(kwargs['prompt'].split()))

241


In [31]:
# Run the inference
result = generate_with_single_input(**kwargs)
print(result['choices'][0]['message']['content'])

You can return the dress within 30 days of delivery. Please ensure that it meets the specific conditions for returns. For more details on the return process, check our return policy.


In [32]:
# Get the total tokens
print(result['usage']['total_tokens'])

341


In [33]:
# GRADED CELL 
def decide_task_nature(query, simplified = True):
    """
    Determines the nature of a query, labeling it as either creative or technical.

    This function constructs a prompt for a language model to decide if a given query requires a creative response,
    such as making suggestions or composing ideas, or a technical response, like providing product details or prices.

    Parameters:
    - query (str): The query to be evaluated for its nature.
    - simplified (bool): If True, uses a simplified prompt.

    Returns:
    - str: The label 'creative' if the query requires creative input, or 'technical' if it requires technical information.
    """


    
    if not simplified:
        PROMPT = f"""Decide if the following query is a query that requires creativity (creating, composing, making new things) or technical (information about products, prices etc.). Label it as creative or technical.
          Examples:
          Give me suggestions on a nice look for a nightclub. Label: creative
          What are the blue dresses you have available? Label: technical
          Give me three Tshirts for summer. Label: technical
          Give me a look for attending a wedding party. Label: creative
          Give me suggestions on clothes that match a green Tshirt. Label: creative
          I would like a suggestion on which products match a green Tshirt I already have. Label: creative

          Query to be analyzed: {query}. Only output one token with the label
          """

    # If simplified, uses a simplified query

    ##############################################
    ######### GRADED PART STARTS HERE ############
    ##############################################

    ### START CODE HERE ###

    else:
        PROMPT = f"""Decide if the following query is a query that requires creativity or technical . Label it as creative or technical.
          Examples:
          Give me suggestions on a nice look for a nightclub. Label: creative
          What are the blue dresses you have available? Label: technical
          Give me three Tshirts for summer. Label: technical

          Query to be analyzed: {query}. Only output one token with the label
          """
    ### END CODE HERE ###

    ##############################################
    ######### GRADED PART ENDS HERE ##############
    ##############################################

    
    with tracer.start_as_current_span("decide_task_nature", openinference_span_kind="tool") as span:
    # Generate the kwards dictionary by passing the PROMPT, low temperature and max_tokens = 1
        span.set_input({"query":query, "simplified": simplified})
        kwargs = generate_params_dict(PROMPT, temperature = 0, max_tokens = 1)

        with tracer.start_as_current_span("router_call", openinference_span_kind = 'llm') as router_span:
            router_span.set_input(kwargs)
            try:
                response = generate_with_single_input(**kwargs) 
            except Exception as error:
                router_span.record_exception(error)
                router_span.set_status(Status(StatusCode.ERROR))
            else:
                # OpenInference Semantic Conventions for computing Costs
                router_span.set_attribute("llm.token_count.prompt", response['usage']['prompt_tokens'])
                router_span.set_attribute("llm.token_count.completion", response['usage']['completion_tokens'])
                router_span.set_attribute("llm.token_count.total", response['usage']['total_tokens'])
                router_span.set_attribute("llm.model_name", response['model'])
                router_span.set_attribute("llm.provider", 'together.ai')
                router_span.set_output(response)
                router_span.set_status(Status(StatusCode.OK))

        # Get the Label by accessing the content key of the response dictionary
        label = response['choices'][0]['message']['content']
        total_tokens = response['usage']['total_tokens']
        span.set_output(str({"label": label, 'total_tokens':total_tokens}))
        span.set_status(Status(StatusCode.OK))    
    
        return label, total_tokens

In [34]:
unittests.test_decide_task_nature(decide_task_nature)

 All tests passed!


In [35]:
queries = ["Give me two sneakers with vibrant colors.",
           "What are the most expensive clothes you have in your catalogue?",
           "I have a green Dress and I like a suggestion on an accessory to match with it.",
           "Give me three trousers with vibrant colors you have in your catalogue.",
           "Create a look for a woman walking in a park on a sunny day. It must be fresh due to hot weather."
           ]

labels = ['technical', 'technical', 'creative', 'technical', 'creative']

In [36]:
for query, correct_label in zip(queries, labels):
    response, total_tokens = decide_task_nature(query, simplified = True)
    label = response
    if label == correct_label:
        label = "\033[32m" + label + "\033[0m" 
    else:
        label = "\033[31m" + label + "\033[0m"
    if total_tokens > 170:
        total_tokens = "\033[31m"  + str(total_tokens) + "\033[0m"
    else:
        total_tokens = "\033[32m"  + str(total_tokens) + "\033[0m"
    print(f"Query: {query} Label Predicted: {label}. Correct Label: {correct_label} Total Tokens: {total_tokens}")

Query: Give me two sneakers with vibrant colors. Label Predicted: creative. Correct Label: technical Total Tokens: 99
Query: What are the most expensive clothes you have in your catalogue? Label Predicted: technical. Correct Label: technical Total Tokens: 103
Query: I have a green Dress and I like a suggestion on an accessory to match with it. Label Predicted: creative. Correct Label: creative Total Tokens: 109
Query: Give me three trousers with vibrant colors you have in your catalogue. Label Predicted: technical. Correct Label: technical Total Tokens: 104
Query: Create a look for a woman walking in a park on a sunny day. It must be fresh due to hot weather. Label Predicted: creative. Correct Label: creative Total Tokens: 115


In [37]:
@tracer.tool
def get_params_for_task(task):
    """
    Retrieves specific language model parameters based on the task nature.

    This function provides parameter sets tailored for creative or technical tasks to optimize
    language model behavior. For creative tasks, higher randomness is encouraged, while technical
    tasks are handled with more focus and precision. A default parameter set is provided for unexpected cases.

    Parameters:
    - task (str): The nature of the task ('creative' or 'technical').

    Returns:
    - dict: A dictionary containing 'top_p' and 'temperature' settings for the specified task.
    """
    # Create the parameters dict for technical and creative tasks
    PARAMETERS_DICT = {"creative": {'top_p': 0.9, 'temperature': 1},
                       "technical": {'top_p': 0.7, 'temperature': 0.3}} 
    
    # If task is technical, return the value for the key technical in PARAMETERS_DICT
    if task == 'technical':
        param_dict = PARAMETERS_DICT['technical'] 

    # If task is creative, return the value for the key creative in PARAMETERS_DICT
    if task == 'creative':
        param_dict = PARAMETERS_DICT['creative'] 

    # If task is a different value, fallback to another set of parameters
    else: # Fallback to a standard value
        param_dict = {'top_p': 0.5, 'temperature': 1} 

    
    return param_dict

In [38]:
# Let's remember the data structure of a product
products_data[0]

{'gender': 'Men',
 'masterCategory': 'Apparel',
 'subCategory': 'Topwear',
 'articleType': 'Shirts',
 'baseColour': 'Navy Blue',
 'season': 'Fall',
 'year': 2011,
 'usage': 'Casual',
 'productDisplayName': 'Turtle Check Men Navy Blue Shirt',
 'price': 67.0,
 'product_id': 15970}

In [39]:
# Run this cell to generate the dictionary with the possible values for each key
values = {}
for d in products_data:
    for key, val in d.items():
        if key in ('product_id', 'price', 'productDisplayName', 'subCategory', 'year'):
            continue
        if key not in values.keys():
            values[key] = set()
        values[key].add(val)

In [40]:
# Example of possible values for the feature 'season'
values['season']

{'All seasons', 'Fall', 'Spring', 'Summer', 'Winter'}

In [41]:
def generate_metadata_from_query(query):
    """
    Generates metadata in JSON format based on a given query to filter clothing items.

    This function constructs a prompt for a language model to create a JSON object that will
    guide the filtering of a vector database query for clothing items. It takes possible values from
    a predefined set and ensures only relevant metadata is included in the output JSON.

    Parameters:
    - query (str): The query describing specific clothing-related needs.

    Returns:
    - str: A JSON string representing metadata with keys like gender, masterCategory, articleType,
      baseColour, price, usage, and season. Each value in the JSON is within a list, with prices specified
      as a dict containing "min" and "max" values. Unrestricted keys should use ["Any"] and unspecified
      prices should default to {"min": 0, "max": "inf"}.
    """

    # Set the prompt. Remember to include the query, the desired JSON format, the possible values (passing {values} at some point) 
    # and explain to the LLM what is going on. 
    # Explicitly tell the llm to include gender, masterCategory, ArticleType, baseColour, price, usage and season as keys.
    # Also mention to the llm that price key must be a json with "min" and "max" values (0 if no lower bound and inf if no upper bound)
    # If there is no price set, add min = 0 and max = inf.
    PROMPT = f"""
    One query will be provided. For the given query, there will be a call on vector database to query relevant clothing items. 
    Generate a JSON with useful metadata to filter the products in the query. Possible values for each feature is in the following json: {values}

    Provide a JSON with the features that best fit in the query (can be more than one, write in a list). Also, if present, add a price key, saying if there is a price range (between values, greater than or smaller than some value).
    Only return the JSON, nothing more. price key must be a JSON with "min" and "max" values (0 if no lower bound and inf if no upper bound). 
    Always include gender, masterCategory, articleType, baseColour, price, usage and season as keys. All values must be within lists.
    If there is no price set, add min = 0 and max = inf.
    Only include values that are given in the json above. 
    
    Example of expected JSON:

    {{
    "gender": ["Women"],
    "masterCategory": ["Apparel"],
    "articleType": ["Dresses"],
    "baseColour": ["Blue"],
    "price": {{"min": 0, "max": "inf"}},
    "usage": ["Formal"],
    "season": ["All seasons"]
    }}

    Query: {query}
             """
    with tracer.start_as_current_span("generate_metadata_from_query", openinference_span_kind="tool") as span:
        span.set_input(query)
        with tracer.start_as_current_span("llm_call", openinference_span_kind="llm") as metadata_span:
            # Generate the response with the generate_with_single_input, PROMPT, temperature = 0 (low randomness) and max_tokens = 1500.
            kwargs = {"prompt": PROMPT, 'temperature': 0, "max_tokens": 1500}  # @REPLACE EQUALS None
            metadata_span.set_input(kwargs)
            try:
                response = generate_with_single_input(**kwargs) 
            except Exception as error:
                metadata_span.record_exception(error)
                metadata_span.set_status(Status(StatusCode.ERROR))
            else:
                # OpenInference Semantic Conventions for computing Costs
                metadata_span.set_attribute("llm.token_count.prompt", response['usage']['prompt_tokens'])
                metadata_span.set_attribute("llm.token_count.completion", response['usage']['completion_tokens'])
                metadata_span.set_attribute("llm.token_count.total", response['usage']['total_tokens'])
                metadata_span.set_attribute("llm.model_name", response['model'])
                metadata_span.set_attribute("llm.provider", 'together.ai')
                metadata_span.set_output(response)
                metadata_span.set_status(Status(StatusCode.OK))

        # Get the Label by accessing the content key of the response dictionary
        content = response['choices'][0]['message']['content']
        total_tokens = response['usage']['total_tokens']
        span.set_output({"content": content, 'total_tokens':total_tokens})
        span.set_status(Status(StatusCode.OK))   

    
    return content, total_tokens

In [42]:
content, total_tokens = generate_metadata_from_query("Create a look for a man that suits a sunny day in the park. I don't want to spend more than 300 dollars on each piece.")

In [52]:
print(content)

{
    "gender": ["Men"],
    "masterCategory": ["Apparel"],
    "articleType": ["Tshirts", "Shorts", "Casual Shoes", "Hats"],
    "baseColour": ["Blue", "White", "Grey", "Tan", "Olive"],
    "price": {"min": 0, "max": 300},
    "usage": ["Casual", "All occasions"],
    "season": ["Summer"]
}


In [53]:
print(total_tokens)

1402


In [49]:

# Set up a batch process with specified fixed size and concurrency
with products_collection.batch.fixed_size(batch_size=20, concurrent_requests=5) as batch:
    # Iterate over a subset of the dataset
    for document in tqdm(products_data[:10]):
        # Generate a UUID based on the chunk text for unique identification
        uuid = generate_uuid5(document['product_id'])

        # Add the chunk object to the batch with properties and UUID
        batch.add_object(
            properties=document,
            uuid=uuid,
        )

100%|█████████████| 10/10 [00:00<00:00, 3094.97it/s]


In [54]:
products_collection = client.collections.get('products')

In [55]:
len(products_collection)

10

In [56]:
@tracer.tool
def parse_json_output(llm_output):
    """
    Parses a string output from a language model into a JSON object.

    This function attempts to clean and parse a JSON-formatted string produced by a language model (LLM).
    The input string might contain minor formatting issues, such as unnecessary newlines or single quotes
    instead of double quotes. The function attempts to correct such issues before parsing.

    Parameters:
    - llm_output (str): The string output from the language model that is expected to be in JSON format.

    Returns:
    - dict or None: A dictionary if parsing is successful, or None if the input string cannot be parsed into valid JSON.

    Exception Handling:
    - In case of a JSONDecodeError during parsing, an error message is printed, and the function returns None.
    """
    try:
        # Since the input might be improperly formatted, ensure any single quotes are removed
        llm_output = llm_output.replace("\n", '').replace("'",'').replace("}}", "}").replace("{{", "{")  # Remove any erroneous structures
        
        # Attempt to parse JSON directly provided it is a properly-structured JSON string
        parsed_json = json.loads(llm_output)
        return parsed_json
    except json.JSONDecodeError as e:
        print(f"JSON parsing failed: {e}")
        return None

@tracer.tool
def get_filter_by_metadata(json_output: dict | None = None):
    """
    Generate a list of Weaviate filters based on a provided metadata dictionary.

    Parameters:
    - json_output (dict) or None: Dictionary containing metadata keys and their values.

    Returns:
    - list[Filter] or None: A list of Weaviate filters, or None if input is None.
    """
    # If the input dictionary is None, return None immediately
    if json_output is None:
        return None

    # Define a tuple of valid keys that are allowed for filtering
    valid_keys = (
        'gender',
        'masterCategory',
        'articleType',
        'baseColour',
        'price',
        'usage',
        'season',
    )

    # Initialize an empty list to store the filters
    filters = []

    # Iterate over each key-value pair in the input dictionary
    for key, value in json_output.items():
        # Skip the key if it is not in the list of valid keys
        if key not in valid_keys:
            continue

        # Special handling for the 'price' key
        if key == 'price':
            # Ensure the value associated with 'price' is a dictionary
            if not isinstance(value, dict):
                continue

            # Extract the minimum and maximum prices from the dictionary
            min_price = value.get('min')
            max_price = value.get('max')

            # Skip if either min_price or max_price is not provided
            if min_price is None or max_price is None:
                continue

            # Skip if min_price is non-positive or max_price is infinity
            if min_price <= 0 or max_price == 'inf':
                continue

            # Add filters for price greater than min_price and less than max_price
            filters.append(Filter.by_property(key).greater_than(min_price))
            filters.append(Filter.by_property(key).less_than(max_price))
        else:
            # For other valid keys, add a filter that checks for any of the provided values
            filters.append(Filter.by_property(key).contains_any(value))

    return filters


@tracer.tool
def generate_filters_from_query(query):
    json_string, total_tokens = generate_metadata_from_query(query)
    json_output = parse_json_output(json_string)
    filters = get_filter_by_metadata(json_output)
    return filters, total_tokens

In [57]:
# GRADED CELL

def get_relevant_products_from_query(query, simplified = False):
    """
    Retrieve the most relevant products for a given query by applying semantic search and optional filters.

    This function generates metadata filters from the query and uses them to search for products 
    that best match the intended criteria. If `simplified` is True, it performs only a basic semantic 
    search with no filters. If the filtered search returns too few results, it progressively reduces 
    filtering constraints based on the predefined importance of each filter.

    Parameters:
    query (str): The query string used to search for relevant products.
    simplified (bool): If True, only a simple semantic search is performed without any metadata filters.

    Returns:
    list: A list of product objects that are most relevant to the query.
    total_tokens: The number of tokens used in the LLM call. Returns 0 if simplified search is used.
    """
    
    ##############################################
    ######### GRADED PART STARTS HERE ############
    ##############################################
    
    ### START CODE HERE ###
    
    # If simplified, just do a semantic search with 20 objects and return it
    if simplified:
        with tracer.start_as_current_span("get_relevant_products_from_query", openinference_span_kind="retriever") as span:  
            span.set_input({'query':query, 'simplified':simplified})
            
            ### YOUR CODE BELOW ###
            results = products_collection.query.near_text(query, limit=20)

            # Set the retrieved documents as attributes on the span
            for i, document in enumerate(results.objects): 
                span.set_attribute(f"retrieval.documents.{i}.document.id", str(document.uuid)) 
                span.set_attribute(f"retrieval.documents.{i}.document.metadata", str(document.metadata)) 
                span.set_attribute( 
                    f"retrieval.documents.{i}.document.content", str(document.properties) #@ KEEP
                )  
    
    ### END CODE HERE ###

    ##############################################
    ######### GRADED PART ENDS HERE #############
    ##############################################
            
            span.set_output({"results": results.objects, "total_tokens": 0})
            span.set_status(Status(StatusCode.OK))  
    
            return results.objects, 0  # Total tokens in this case is 0 because there was no LLM call!
    # If not simplified, perform the previous workflow by generating the filters and then doing a semantic search with them
    
    with tracer.start_as_current_span("get_relevant_products_from_query", openinference_span_kind="retriever") as span:  
        span.set_input({'query':query, 'simplified':simplified})
        filters, total_tokens = generate_filters_from_query(query)  # Generate filters based on the query

    # Check if there are no applicable filters
        if filters is None or len(filters) == 0:
            span.set_attribute("retrieval.filters", '')
            results = products_collection.query.near_text(query, limit=20) 
            # Set the retrieved documents as attributes on the span
            for i, document in enumerate(results.objects): 
                span.set_attribute(f"retrieval.documents.{i}.document.id", str(document.uuid))
                span.set_attribute(f"retrieval.documents.{i}.document.metadata", str(document.metadata)) 
                span.set_attribute( 
                    f"retrieval.documents.{i}.document.content", str(document.properties) 
                )  
            span.set_output({"results": results.objects, "total_tokens": total_tokens})
            span.set_status(Status(StatusCode.OK))  
            return results.objects, total_tokens
    # Query with filters and limit to the top 20 relevant objects
        span.set_attribute("retrieval.filters", str(filters))
        results = products_collection.query.near_text(query, filters=Filter.all_of(filters), limit=20)
        span.set_attribute("retrieval.len", len(results.objects))
        # Set the retrieved documents as attributes on the span
        for i, document in enumerate(results.objects): 
            span.set_attribute(f"retrieval.documents.{i}.document.id", str(document.uuid))
            span.set_attribute(f"retrieval.documents.{i}.document.metadata", str(document.metadata)) 
            span.set_attribute( 
                f"retrieval.documents.{i}.document.content", str(document.properties) 
            )
    
        # If the result set contains fewer than 10 products, try reducing filters to broaden the search
        importance_order = [ 'baseColour', 'masterCategory', 'usage', 'masterCategory', 'season', 'articleType', 'gender']
        if len(results.objects) < 10:
            # Iterate through the importance order of filters
            for i in range(len(importance_order)):
                with tracer.start_as_current_span(f"refilter_{i}", openinference_span_kind="chain") as refilter_span: 
                    # Create a list of filters that excludes less important ones
                    filtered_filters = [x for x in filters if x.target in importance_order[i+1:]]
                    refilter_span.set_input(str(filtered_filters))
                    
                    results = products_collection.query.near_text(query, filters=Filter.all_of(filtered_filters), limit=20)
                    # Set the retrieved documents as attributes on the span
                    for j, document in enumerate(results.objects): 
                        refilter_span.set_attribute(f"retrieval.documents.{j}.document.id", str(document.uuid))
                        refilter_span.set_attribute(f"retrieval.documents.{j}.document.metadata", str(document.metadata)) 
                        refilter_span.set_attribute( 
                            f"retrieval.documents.{j}.document.content", str(document.properties) 
                        )
                    # If sufficient products have been found, return early
                    if len(results.objects) >= 5:
                        refilter_span.set_output(results.objects)
                        refilter_span.set_status(Status(StatusCode.OK))  
                        span.set_output(results.objects)
                        span.set_status(Status(StatusCode.OK)) 
                        return results.objects, total_tokens
        span.set_output(results.objects)
        span.set_status(Status(StatusCode.OK)) 
        return results.objects, total_tokens  # Return the final set of relevant products

In [58]:
query = "Give me three T-shirts to use in sunny days"

In [59]:
t, total_tokens = get_relevant_products_from_query(query)

In [60]:
total_tokens

1392

In [61]:
t, total_tokens = get_relevant_products_from_query(query, simplified = True)

In [62]:
total_tokens

0

In [63]:
# Test your solution!
unittests.test_get_relevant_products_from_query(get_relevant_products_from_query)

Failed test case: Incorrect result for query = Give me three Tshirts to use in sunny days and simplified = True.
Expected: Product IDs must be {3328, 35983, 54935, 6939, 33565, 49964, 2863, 2866, 1844, 1845, 1846, 1847, 1853, 9539, 1866, 4298, 1867, 3431, 37608, 3318}
Got: Product IDs output are: {15970, 21379, 53759, 59263, 26960, 29114, 30805, 30039, 39386, 1855}




In [64]:
@tracer.tool
def generate_items_context(results):
    """
    Compile detailed product information from a list of result objects into a formatted string.

    Parameters:
    results (list): A list of result objects, each having a `properties` attribute that is a dictionary 
                    containing product attributes such as 'product_id', 'productDisplayName', 
                    'masterCategory', 'usage', 'gender', 'articleType', 'subCategory', 
                    'baseColour', 'season', and 'year'.

    Returns:
    str: A multi-line string where each line contains the formatted details of a single product.
         Each product detail includes the product ID, name, category, usage, gender, type, color, 
         season, and year.
    """
    t = ""  # Initialize an empty string to accumulate product information

    for item in results:  # Iterate through each item in the results list
        item = item.properties  # Access the properties dictionary of the current item

        # Append formatted product details to the output string
        t += (
            f"Product ID: {item['product_id']}. "
            f"Product name: {item['productDisplayName']}. "
            f"Product Category: {item['masterCategory']}. "
            f"Product usage: {item['usage']}. "
            f"Product gender: {item['gender']}. "
            f"Product Type: {item['articleType']}. "
            f"Product Category: {item['subCategory']} "
            f"Product Color: {item['baseColour']}. "
            f"Product Season: {item['season']}. "
            f"Product Year: {item['year']}.\n"
        )

    return t  # Return the complete formatted string with product details

In [65]:
print(generate_items_context(t)[:1000])

Product ID: 15970. Product name: Turtle Check Men Navy Blue Shirt. Product Category: Apparel. Product usage: Casual. Product gender: Men. Product Type: Shirts. Product Category: Topwear Product Color: Navy Blue. Product Season: Fall. Product Year: 2011.
Product ID: 39386. Product name: Peter England Men Party Blue Jeans. Product Category: Apparel. Product usage: Casual. Product gender: Men. Product Type: Jeans. Product Category: Bottomwear Product Color: Blue. Product Season: Summer. Product Year: 2012.
Product ID: 30805. Product name: Fabindia Men Striped Green Shirt. Product Category: Apparel. Product usage: Ethnic. Product gender: Men. Product Type: Shirts. Product Category: Topwear Product Color: Green. Product Season: Summer. Product Year: 2012.
Product ID: 26960. Product name: Jealous 21 Women Purple Shirt. Product Category: Apparel. Product usage: Casual. Product gender: Women. Product Type: Shirts. Product Category: Topwear Product Color: Purple. Product Season: Summer. Product

In [66]:
@tracer.tool
def query_on_products(query, simplified = False):
    """
    Execute a product query process to generate a response based on the nature of the query.

    Parameters:
    query (str): The input query string that needs to be analyzed and answered using product data.
    task_nature_prompt_function (func): The prompt function to be used to decide the task nature (if creative of technical)
    simplified (bool): If True, does not use LLM to generate metadata for filtering

    Returns:
    dict: A dictionary of keyword arguments (`kwargs`) containing the prompt and additional settings 
          for creating a response, suitable for input to an LLM or other processing system.
    int: Number of tokens used in the process to create the kwargs dictionary

    Outputs:
    str: The content of the generated response from the LLM based on the provided query and product 
         information.
    """
    total_tokens = 0
    # Determine if the query is technical or creative in nature
    
    query_label, tokens = decide_task_nature(query, simplified = simplified)
    
    # Sum the tokens used to decide the task nature (creative or technical)
    total_tokens += tokens

    # Obtain necessary parameters based on the query type
    parameters_dict = get_params_for_task(query_label)
    
    # Retrieve products that are relevant to the query
    relevant_products, tokens = get_relevant_products_from_query(query, simplified = simplified)
    
    # Sum the tokens used to get relevant products 
    total_tokens += tokens
     
    # Create a context string from the relevant products
    context = generate_items_context(relevant_products)

    # Construct a prompt including product details and the query. Remember to add the context and the query in the prompt, also, ask the LLM to provide the product ID in the answer
    PROMPT = (
        f"Given the available set of clothing products given by: "
        f"CLOTHING PRODUCTS AVAILABLE:\n{context}\n"
        f"Answer the question that follows.\n"
        f"Never use more than 5 clothing products available below to compose your answer.\n"
        f"Provide the item ID in your answers.\n"
        f"The other information might be provided but not necessarily all of them, pick only the relevant ones for the given query.\n"
        
        f"QUERY: {query}"
    )
    
    # Generate kwargs (parameters dict) for parameterized input to the LLM with , Prompt, role = 'assistant' and **parameters_dict
    kwargs = generate_params_dict(PROMPT, role='assistant', **parameters_dict) 

    
    return kwargs, total_tokens

In [67]:
kwargs, total_tokens = query_on_products('Make a wonderful look for a man attending a wedding party happening during night.', simplified = False)

In [68]:
result = generate_with_single_input(**kwargs)
print(result['choices'][0]['message']['content'])

For a man attending a wedding party at night, I recommend the following outfit to create a stylish look:

1. **Product ID: 30805** - Fabindia Men Striped Green Shirt: This shirt adds an elegant touch with its ethnic stripes.
2. **Product ID: 39386** - Peter England Men Party Blue Jeans: These jeans provide a perfect balance of comfort and style for a semi-formal setting.
3. **Product ID: 30039** - Skagen Men Black Watch: A sleek watch adds a sophisticated element to the overall look.

This combination is both stylish and appropriate for a wedding party at night.


In [69]:
print(f"Total tokens used in the query is: {total_tokens + result['usage']['total_tokens']}")

Total tokens used in the query is: 2262


In [70]:
kwargs, total_tokens = query_on_products('Make a wonderful look for a man attending a wedding party happening during night.', simplified = True)

In [71]:
total_tokens

106

In [72]:
result = generate_with_single_input(**kwargs)
print(result['choices'][0]['message']['content'])

For a man attending a wedding party during the night, a great look could include:

1. **Product ID: 15970** - Turtle Check Men Navy Blue Shirt (Casual, Topwear)
2. **Product ID: 39386** - Peter England Men Party Blue Jeans (Casual, Bottomwear)
3. **Product ID: 30039** - Skagen Men Black Watch (Casual, Accessories)

This combination of a navy blue shirt paired with stylish blue jeans, along with a black watch for a touch of elegance, will create a sophisticated and trendy look for the evening event.


In [73]:
print(f"Total tokens used in the query is: {total_tokens + result['usage']['total_tokens']}")

Total tokens used in the query is: 914


In [75]:
@tracer.tool
def answer_query(query, model = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",simplified=False):
    """
    Processes a user's query to determine its type (FAQ or Product) and executes the appropriate workflow.
    
    Parameters:
    - query (str): The query string provided by the user.
    - model (str): The model that will answer the question. Defaults to meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo'
    - simplified (bool): If True, uses a simplified version of the method. Defaults to False.
    
    Returns:
    - dict: A dictionary containing keyword arguments for further processing.
      If the query is neither FAQ nor Product-related, returns a default response dictionary instructing
      the assistant to answer based on existing context.
    """
    # Initialize the total tokens used to zero
    total_tokens = 0
    
    # Determine if the query is FAQ or Product and get the token count for this step
    label, tokens = check_if_faq_or_product(query, simplified=simplified)
    
    # Sum the tokens
    total_tokens += tokens
    
    # If the query is neither FAQ nor Product, return a default response
    if label not in ['FAQ', 'Product']:
        return {
            "role": "assistant",
            "prompt": (f"User provided a question that does not fit FAQ or Product-related categories. "
                       f"Answer it based on the context you already have. Query provided by the user: {query}")
        }
    
    # Process the query based on its label
    if label == 'FAQ':
        # Handle FAQ-related queries
        kwargs = query_on_faq(query, simplified=simplified)
    elif label == 'Product':
        try:
            # Handle Product-related queries, with error handling in place
            kwargs, tokens = query_on_products(query, simplified=simplified)
            # Add the tokens to the total tokens
            total_tokens += tokens
        except Exception:
            # Return an error response if an exception occurs during querying
            return {
                "role": "assistant",
                "prompt": (f"User provided a question that broke the querying system. "
                           f"Instruct them to rephrase it. Answer it based on the context you already have. "
                           f"Query provided by the user: {query}")
            }, total_tokens
    # Set the model to answer the final query - usually a better one         
    # kwargs['model'] = model
    # Return the kwargs and total_tokens for further processing
    return kwargs, total_tokens

In [76]:
kwargs, total_tokens = answer_query("Give me three examples of blue t-shirts available on your catalogue.", simplified = False)

In [77]:
result = generate_with_single_input(**kwargs)
print(result['choices'][0]['message']['content'])

I'm sorry, but there are no blue t-shirts available in the provided catalogue. The closest options include a navy blue shirt (Product ID: 15970) and a grey t-shirt (Product ID: 53759 and Product ID: 1855), but none are specifically blue t-shirts.


In [78]:
# To get the total tokens for the call, we must sum the total_tokens to get the kwargs dictionary + total tokens from the LLM call
total_tokens +  result['usage']['total_tokens']

2482

In [79]:
kwargs, total_tokens = answer_query("Give me three examples of blue t-shirts available on your catalogue.", simplified = True)

In [80]:
result = generate_with_single_input(**kwargs)
print(result['choices'][0]['message']['content'])

Here are three examples of blue t-shirts available in the catalogue:

1. Product ID: 39386 - Peter England Men Party Blue Jeans (Note: This is actually jeans, not a t-shirt, but it's included as a blue item).
2. Product ID: 53759 - Puma Men Grey T-shirt (Note: This is grey, not blue, but it's included as a t-shirt).
3. Unfortunately, there are no specific blue t-shirts listed in the provided products.

It seems there are no blue t-shirts available in the current catalogue. Would you like to see other types of blue clothing?


In [81]:
total_tokens +  result['usage']['total_tokens']

1010

In [82]:
chat_widget_standard = ChatWidget(generator_function = lambda x: answer_query(x, simplified = False), tracer = tracer)

In [83]:
make_url()

FOLLOW THIS URL TO OPEN THE UI: http://127.0.0.1:8888


In [84]:
chat_widget_simplified = ChatWidget(generator_function = lambda x: answer_query(x, simplified = True), tracer = tracer)

In [85]:
make_url()

FOLLOW THIS URL TO OPEN THE UI: http://127.0.0.1:8888
